In [ ]:
from pathlib import Path

import numpy as np

import spikeinterface.full as si
import probeinterface as pi
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from spikeinterface.core import NumpySorting, BinaryRecordingExtractor

In [ ]:
# make initial large binary file, combining all relevant data

data_folder = Path('/home/nolanlab/Work/Ale_Project/data/')
pathnames = ['2025_05_02_Burst-RS-02.npz', '2025_05_02_Burst-RS-03.npz', '2025_05_02_Burst-RS-05.npz', '2025_05_07_Burst-RS-05.npz']

num_channels = len(pathnames)

data_files = [data_folder / filename for filename in pathnames]
all_traces = np.array([np.load(data_file)['trace_roi_raw'] for data_file in data_files])
all_traces.tofile(data_folder / 'all_recordings.bin', format='float64')

In [ ]:
# Make initial recording and probe geometry

binary_filepath = data_folder / 'all_recordings.bin'
rec_groups = BinaryRecordingExtractor(binary_filepath, sampling_frequency=1683, dtype='float64', num_channels=num_channels, channel_ids=pathnames)

big_probe = pi.Probe(ndim=2, si_units='um')
big_probe.set_contacts(positions=[[0,100*a] for a in range(num_channels)])
big_probe.set_device_channel_indices(range(num_channels))
rec_groups = rec_groups.set_probe(big_probe)
rec_groups.set_property('group', range(num_channels))


In [ ]:
# Do the peak detection

filtered_recording = si.highpass_filter(rec_groups, freq_min=100)

spikes = {}
for rec_id, rec in filtered_recording.split_by('group').items():

    # do simple peak detection
    simple_peaks = detect_peaks(rec, method='by_channel', method_kwargs={'detect_threshold': 5, 'exclude_sweep_ms': 1.2})

    # compute template for template matching
    sort = NumpySorting.from_unit_dict({0: simple_peaks['sample_index']}, sampling_frequency=rec.sampling_frequency)
    temp_sa = si.create_sorting_analyzer(sort, rec)
    temp_sa.compute({'random_spikes':{}, 'templates':{}, 'waveforms': {'ms_before': 5.0, 'ms_after': 10.0}})
    prototype = temp_sa.get_extension('templates').get_data()[0].transpose()[0]

    # do template matching
    # the exclude_sweep_ms needs to be at least one sample. In our case that is 1000/1683 = 0.59
    spikes[rec_id] = detect_peaks(rec, method='matched_filtering', method_kwargs={'detect_threshold': 4.5, 'prototype': np.array(prototype), 'ms_before': 8/24*15, 'exclude_sweep_ms': 0.6})['sample_index']
    
# make sorting objects from just the template matched detected peaks
sort = NumpySorting.from_unit_dict(spikes, sampling_frequency=rec.sampling_frequency)
sort.set_property('group', range(num_channels))

In [ ]:
1000/rec.sampling_frequency

In [ ]:
# Make and analyzer object and analyze

sa = si.create_sorting_analyzer(sort, filtered_recording, method='by_property', by_property = 'group')
sa.compute(
{
    'unit_locations': {},
    'random_spikes': {},
    'noise_levels': {},
    'templates': {},
    'waveforms': {'ms_before': 5.0, 'ms_after': 10.0},
    'spike_amplitudes': {},
    'isi_histograms': {},
    'amplitude_scalings': {},
    'correlograms': {},
    'quality_metrics' : {},
    'template_metrics': {'include_multi_channel_metrics': False},
})

sa.save_as(format='binary_folder', folder='all_sortings')